# **ALARAJOY Library Conversion QA Notebook**


This Jupyter Notebook is designed to enable comparisons for neutron activation responses of a given single parent element as calculated by ALARA for the purpose of validating ALARAJOY-processed data from the FENDL3.2x data sets.

**Import Packages**

In [32]:
import alarajoy_QA as qa
import sys
sys.path.append('..')
import alara_output_processing as aop
import importlib

**Run ALARA with each prepared binary data library**

In [33]:
importlib.reload(qa)
data_source = {
    'fendl2' : '/groupspace/shared/n/nukecode/ALARA/data/fendl2bin', # Example path
    'fendl3' : '../../examples/data/fendl3' # Example path
}

element = input('Select single parent element to evaluate: ').lower()

for libname, binary in data_source.items():
    alara_input = qa.fill_alara_template(element, binary)
    qa.write_alara_input_file(alara_input)
    output = qa.run_alara(element,libname)
    data_source[libname] = output

**Convert ALARA output files to DataFrames**

Store dataframes with each table's metadata into a nested dictionary for all tables in both simulations.

This nested dictionary is of the form:

```
df_dict = {
    'Data Source' : (Either 'fendl2' or 'fendl3'),
    'Variable'    : (Any ALARA output variable, dependent on ALARA
                        run parameters),
    'Unit'        : (Respective unit matching the above variable),
    'Data'        : (DataFrame containing ALARA output data for
                        the given data source, variable, and unit)
}
```

In [34]:
importlib.reload(aop)
dfs = aop.DataProcessing.process_data(data_source)

key_list = list(dfs.keys())
arbitrary_df = dfs[key_list[0]]['Data'] # 0 arbitrarily chosen as key_list index
df_time_cols = arbitrary_df.columns[1:] # All dfs have same time cols; any index valid
times = list(df_time_cols)

# Example DataFrame
dfs['fendl3 Number Density']['Data']

,isotope,shutdown,1e-05y,0.01y,1y,100y,10000y
0,h-1,1.170900e+22,1.170900e+22,1.170900e+22,1.170900e+22,1.170900e+22,1.170900e+22
1,h-2,2.823400e+20,2.823400e+20,2.823400e+20,2.823400e+20,2.823400e+20,2.823400e+20
2,h-3,1.620800e+17,1.620800e+17,1.619800e+17,1.532500e+17,5.979000e+14,0.000000e+00
3,he-3,1.086300e+16,1.086300e+16,1.095400e+16,1.969300e+16,1.723400e+17,1.729400e+17
4,he-4,2.652700e+21,2.652700e+21,2.652700e+21,2.652700e+21,2.652700e+21,2.652700e+21
...,...,...,...,...,...,...,...
219,fe-58r,1.241400e+18,1.241400e+18,1.241400e+18,1.241400e+18,1.241400e+18,1.241400e+18
220,fe-58s,1.627600e+18,1.627600e+18,1.627600e+18,1.627600e+18,1.627600e+18,1.627600e+18
221,fe-58t,4.993900e+17,4.993900e+17,4.993900e+17,4.993900e+17,4.993900e+17,4.993900e+17
222,fe-58u,1.755000e+18,1.755000e+18,1.755000e+18,1.755000e+18,1.755000e+18,1.755000e+18
